In [2]:
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns  
from linares_plot import *

In [41]:
frames=[]

Trucos=['Truco 1', 'Truco 2', 'Truco 3', 'Truco 4', 'Truco 5', 'Truco 6', 'Truco 7', ]


Df=pd.read_excel(os.getcwd() + '\\ANOTACIONES_.xlsx', sheet_name='Truco 6')
subj_idx = Df.columns[2:]


for s in subj_idx:
    df_x = Df[['Tiempo_s', s]]
    df_x.columns=['time', 'labels']
    df_x['subject']=s
    frames.append(df_x)

df = pd.concat(frames, ignore_index=True)

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [42]:
Df

,Tiempo_s,Tiempo,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
0,1286,21:26:00,N,NaN,O,Si,NaN,NaN,I,D,...,NaN,NaN,S,"N, S",NaN,N,Si,NaN,NaN,N
1,1288,21:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,...,NaN,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1290,21:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1292,21:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Se,NaN,NaN,O,NaN,NaN,NaN
4,1294,21:34:00,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN
5,1296,21:36:00,NaN,F,NaN,NaN,NaN,NaN,C,NaN,...,NaN,NaN,"N, C",NaN,C,NaN,NaN,NaN,NaN,NaN
6,1298,21:38:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,O,N,NaN,NaN
7,1300,21:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1302,21:42:00,NaN,NaN,"C, S",NaN,R,S,S,"S, C",...,NaN,"N, S","N, S",NaN,C,"N, C",NaN,NaN,NaN,NaN
9,1304,21:44:00,O,"Se, S",NaN,S,NaN,NaN,NaN,NaN,...,NaN,NaN,S,S,NaN,NaN,NaN,NaN,S,S


In [29]:
df

,time,labels,subject
0,0,0,1
1,2,0,1
2,4,0,1
3,6,0,1
4,8,Se,1
5,10,B,1
6,12,0,1
7,14,0,1
8,16,0,1
9,18,0,1


In [28]:
#df['labels']=df['labels'].replace(df.labels.iloc[0], 0) ##in case I have to get rid of the nans and put 0s


In [11]:
Df=pd.read_excel(os.getcwd() + '\\ANOTACIONES_.xlsx', sheet_name='Truco 1')
Df

,Tiempo_s,Tiempo,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,50
0,0,00:00:00,NaN,AA,A,A,A+,A,A,"S,A",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,00:06:00,NaN,B,NaN,NaN,NaN,NaN,P,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,00:08:00,Se,NaN,NaN,NaN,NaN,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10,00:10:00,B,NaN,NaN,O,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12,00:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,14,00:14:00,NaN,NaN,NaN,NaN,NaN,P,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,16,00:16:00,NaN,NaN,NaN,NaN,NaN,NaN,C,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,18,00:18:00,NaN,NaN,NaN,B,NaN,NaN,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
Df.Tiempo

0      00:00:00
1      00:02:00
2      00:04:00
3      00:06:00
4      00:08:00
5      00:10:00
6      00:12:00
7      00:14:00
8      00:16:00
9      00:18:00
10     00:20:00
11     00:22:00
12     00:24:00
13     00:26:00
14     00:28:00
15     00:30:00
16     00:32:00
17     00:34:00
18     00:36:00
19     00:38:00
20     00:40:00
21     00:42:00
22     00:44:00
23     00:46:00
24     00:48:00
25     00:50:00
26     00:52:00
27     00:54:00
28     00:56:00
29     00:58:00
         ...   
266         NaN
267         NaN
268         NaN
269         NaN
270         NaN
271         NaN
272         NaN
273         NaN
274         NaN
275         NaN
276         NaN
277         NaN
278         NaN
279         NaN
280         NaN
281    09:22:00
282    09:24:00
283    09:26:00
284    09:28:00
285    09:30:00
286    09:32:00
287    09:34:00
288    09:36:00
289    09:38:00
290    09:40:00
291    09:42:00
292    09:44:00
293    09:46:00
294    09:48:00
295    09:50:00
Name: Tiempo, Length: 29